In [1]:
from audiomentations import AddBackgroundNoise, PolarityInversion
from imutils import paths
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
import numpy as np
import librosa
import random
import shutil
import json
import os 

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
DATASET_PATH = 'Tatar speech commands/'

metadata = pd.read_excel(os.path.join(DATASET_PATH, 'Tatar commands.xlsx'))
metadata.head()

,Google's Speech Commands V2,Translated (tt),Folder name,Number of recordings
0,backward,артка,4866,96
1,forward,алга,4867,98
2,right,уңга,4868,101
3,left,сулга,4869,103
4,down,аска,4870,100


In [3]:
command_map = {}
for ind in metadata.index:
    command_id = metadata['Folder name'][ind]
    command_name = metadata['Google\'s Speech Commands V2'][ind]
    command_map[command_id] = command_name
    print(ind, command_id, command_name)

0 4866 backward
1 4867 forward
2 4868 right
3 4869 left
4 4870 down
5 4871 up
6 4873 go
7 4872 stop
8 4874 on
9 4875 off
10 4876 yes
11 4877 no
12 4878 learn
13 4879 follow
14 4880 zero
15 4881 one
16 4882 two
17 4883 three
18 4884 four
19 4885 five
20 4886 six
21 4887 seven
22 4888 eight
23 4889 nine
24 4890 bed
25 4891 bird
26 4892 cat
27 4893 dog
28 4894 happy
29 4895 house
30 4896 read
31 4897 write
32 4898 tree
33 4899 visual
34 4900 wow


In [4]:
# path to save the processed dataset
new_dataset_path = "Keyword-MLP/data_tt/"
# path to the background noise dataset
bn_dataset_path = "ESC-50-master/audio" 

n = 30 # number of randomly selected test subjects
    
commands = ["backward", "forward", "right", "left", "down", "up", "go", "stop", "on", "off", "yes", "no", 
            "learn", "follow", "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", 
            "nine", "bed", "bird", "cat", "dog", "happy", "house", "read", "write", "tree", "visual", "wow"]

In [5]:
# create new directories
for command in commands:
    command_path = os.path.join(new_dataset_path, command)
    if not os.path.exists(command_path):
        os.makedirs(command_path)

In [6]:
# check the num of speech commands in KSCD
wavPaths = list(paths.list_files(DATASET_PATH, validExts="mp3"))
print(len(wavPaths))

3547


In [7]:
for key, val in command_map.items():
    mp3_files = list(paths.list_files(os.path.join(DATASET_PATH,str(key)), validExts="mp3"))
    print(val, len(mp3_files))

backward 96
forward 98
right 101
left 103
down 100
up 101
go 102
stop 104
on 102
off 100
yes 103
no 103
learn 100
follow 97
zero 102
one 101
two 100
three 100
four 100
five 104
six 101
seven 103
eight 100
nine 99
bed 98
bird 101
cat 104
dog 104
happy 103
house 104
read 104
write 104
tree 102
visual 103
wow 100


In [8]:
# create a dictionary
# key: subject ID
# value: num of samples 
dic = {}

for wavPath in wavPaths:
    wav_name = wavPath.split('/')[-1]
    wav_name = wav_name.split('.')[0]
    #print(wavPath)
    #print(wav_name.split('_'))
    cmd_id, sub_id, *rest = wav_name.split('_')
    if sub_id in dic:
        dic[sub_id] += 1
    else:
        dic[sub_id] = 1

len(dic)

153

In [9]:
# split the subjects into
# training, validation, and test set
train_subjects = []
test_subjects = []
val_subjects = []

for key, val in dic.items():
    if val == 35:
        test_subjects.append(key)
    else:
        train_subjects.append(key)

val_subjects = test_subjects[n:n+10]
train_subjects += test_subjects[n+10:]
test_subjects = test_subjects[:n]


print("# training subjects:", len(train_subjects))
print("# validation subjects:", len(val_subjects))
print("# testing subjects:", len(test_subjects))

# training subjects: 113
# validation subjects: 10
# testing subjects: 30


In [25]:
wavPaths = list(paths.list_files(DATASET_PATH, validExts="mp3"))

train_cnt = 0
val_cnt = 0
test_cnt = 0
for wavPath in wavPaths:
    sub_id = wavPath.split('/')[-1].split('.')[0]
    sub_id = sub_id.split('_')[1]
    if sub_id in train_subjects:
        train_cnt += 1
    elif sub_id in val_subjects:
        val_cnt += 1
    else:
        test_cnt += 1
        
print(train_cnt, val_cnt, test_cnt)

2147 350 1050


In [10]:
sample_rate = 16000 # sampling rate in kHz
duration = 1 # audio length in seconds
speeds = [0.8, 1, 1.2] # time stretch rates
pitches = [-2, 0, 2] # pitch scales
gain_min_factor, gain_max_factor  = 0.8, 1.2 # min-max gain factors
noise_percentage_factor = 0.05 # white gaussian noise 
num_background_noise = 2 # number of background noises 

In [11]:
# https://github.com/musikalkemist/audioDataAugmentationTutorial/blob/main/3/dataaugmentation.py
def add_white_noise(signal, noise_percentage_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_percentage_factor
    return augmented_signal


def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(y=signal, rate=time_stretch_rate)


def pitch_scale(signal, sample_rate, num_semitones):
    return librosa.effects.pitch_shift(y=signal, sr=sample_rate, n_steps=num_semitones)


def random_gain(signal, min_factor, max_factor):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal


def invert_polarity(signal):
    return signal * -1


add_background_noise = AddBackgroundNoise(sounds_path=bn_dataset_path,
                                          min_snr_in_db=5.0,
                                          max_snr_in_db=30.0,
                                          noise_transform=PolarityInversion(),
                                          p=1.0)

In [12]:
# convert lists to sets for fast searching
train_subjects = set(train_subjects)
test_subjects = set(test_subjects)
val_subjects = set(val_subjects)

In [13]:
# create training, validation, and test lists
train_list = []
val_list = []
test_list = []

# loop over the speech commands
for i, wavPath in enumerate(wavPaths, 1):
    print("[INFO] Processing file: {}/{}".format(i, len(wavPaths)))
    
    # extract the command name, wav file name, and subject ID
    wav_file = wavPath.split('/')[-1]
    wav_file = wav_file.split('.')[0]
    command_id, subject, *rest = wav_file.split('_')
    command = command_map[int(command_id)]
    print(command_id, command, subject)
    #command, wav_file = wavPath.split('/')[-2:]
    #subject = wav_file.split('.')[0]
    
    # load the raw audio signal
    raw_signal, _ = librosa.load(wavPath, sr=sample_rate)
    
    # define a path for saving the augmented command
    save_path = os.path.join(new_dataset_path, command)
    
    if subject in train_subjects or subject in val_subjects:
        # loop over the time stretch rates
        for speed in speeds:
            # apply time stretch augmentation 
            ts_signal = time_stretch(raw_signal, speed)     

            # loop over the pitch scales
            for pitch in pitches:
                # apply pitch scale augmentation
                ps_signal = pitch_scale(ts_signal, sample_rate, pitch)
                
                # apply gain scale augmentation
                gs_signal = random_gain(ps_signal, gain_min_factor, gain_max_factor)
                wav_file_name = "{}-{}-{}-0.wav".format(subject, speed, pitch)
                sf.write(os.path.join(save_path, wav_file_name), gs_signal, sample_rate)
                
                if subject in train_subjects:
                    train_list.append('./data_tt/' + command + '/' + wav_file_name)
                else:
                    val_list.append('./data_tt/' + command + '/' + wav_file_name)
                
                # add white gaussian noise 
                gn_signal = add_white_noise(gs_signal, noise_percentage_factor)
                gn_wav_file_name = "{}-{}-{}-1.wav".format(subject, speed, pitch)
                sf.write(os.path.join(save_path, gn_wav_file_name), gn_signal, sample_rate)
                
                if subject in train_subjects:
                    train_list.append('./data_tt/' + command + '/' + gn_wav_file_name)
                else:
                    val_list.append('./data_tt/' + command + '/' + gn_wav_file_name)
                    
                # add random background noise
                for i in range(num_background_noise):
                    bn_signal = add_background_noise(gs_signal, sample_rate=sample_rate)
                    bn_wav_file_name = "{}-{}-{}-{}.wav".format(subject, speed, pitch, i+2)
                    sf.write(os.path.join(save_path, bn_wav_file_name), bn_signal, sample_rate)
                    
                    if subject in train_subjects:
                        train_list.append('./data_tt/' + command + '/' + bn_wav_file_name)
                    else:
                        val_list.append('./data_tt/' + command + '/' + bn_wav_file_name)
    else:
        sf.write(os.path.join(save_path, wav_file+'.wav'), raw_signal, sample_rate)
        test_list.append('./data_tt/' + command + '/' + wav_file+'.wav')

[INFO] Processing file: 1/3547
4880 zero 516721534
[INFO] Processing file: 2/3547
4880 zero 1725267798
[INFO] Processing file: 3/3547
4880 zero 335145783


/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/5-200334-B-1.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/5-177614-A-5.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/2-152895-A-31.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 4/3547
4880 zero 390959255
[INFO] Processing file: 5/3547
4880 zero 835724583
[INFO] Processing file: 6/3547
4880 zero 1340570009
[INFO] Processing file: 7/3547
4880 zero 319520127


/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/3-164595-A-15.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/2-103427-A-3.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/4-264453-A-34.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 8/3547
4880 zero 1975608245
[INFO] Processing file: 9/3547
4880 zero 1371986321
[INFO] Processing file: 10/3547
4880 zero 262192500


...

[INFO] Processing file: 3141/3547
4887 seven 1060745078
[INFO] Processing file: 3142/3547
4887 seven 609129547
[INFO] Processing file: 3143/3547
4887 seven 1763145949
[INFO] Processing file: 3144/3547
4887 seven 2129135435
[INFO] Processing file: 3145/3547
4887 seven 1340570009
[INFO] Processing file: 3146/3547
4890 bed 1164979260
[INFO] Processing file: 3147/3547
4890 bed 808985421
[INFO] Processing file: 3148/3547
4890 bed 5230223819
[INFO] Processing file: 3149/3547
4890 bed 1884416453
[INFO] Processing file: 3150/3547
4890 bed 308796398
[INFO] Processing file: 3151/3547
4890 bed 5277261547
[INFO] Processing file: 3152/3547
4890 bed 125356271
[INFO] Processing file: 3153/3547
4890 bed 170655814
[INFO] Processing file: 3154/3547
4890 bed 1231589731
[INFO] Processing file: 3155/3547
4890 bed 1218613958
[INFO] Processing file: 3156/3547
4890 bed 609129547
[INFO] Processing file: 3157/3547
4890 bed 5186511260
[INFO] Processing file: 3158/3547
4890 bed 65867193
[INFO] Processing file: 31

/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/2-130979-A-21.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 3348/3547
4895 house 249534318
[INFO] Processing file: 3349/3547
4895 house 1121086406
[INFO] Processing file: 3350/3547
4895 house 1237981430
[INFO] Processing file: 3351/3547
4895 house 1718497822
[INFO] Processing file: 3352/3547
4895 house 1219900046
[INFO] Processing file: 3353/3547
4895 house 266884751
[INFO] Processing file: 3354/3547
4895 house 1188788203
[INFO] Processing file: 3355/3547
4895 house 835724583
[INFO] Processing file: 3356/3547
4895 house 262192500
[INFO] Processing file: 3357/3547
4895 house 609129547
[INFO] Processing file: 3358/3547
4895 house 2104105514
[INFO] Processing file: 3359/3547
4895 house 268175952
[INFO] Processing file: 3360/3547
4895 house 250582507
[INFO] Processing file: 3361/3547
4895 house 834995101
[INFO] Processing file: 3362/3547
4895 house 5080096538
[INFO] Processing file: 3363/3547
4895 house 1036424300
[INFO] Processing file: 3364/3547
4895 house 1355297933
[INFO] Processing file: 3365/3547
4895 house 722174255
[

/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/4-130584-A-4.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 3470/3547
4870 down 1064144631
[INFO] Processing file: 3471/3547
4870 down 518430441
[INFO] Processing file: 3472/3547
4870 down 1164979260
[INFO] Processing file: 3473/3547
4870 down 1032786543
[INFO] Processing file: 3474/3547
4870 down 443528311
[INFO] Processing file: 3475/3547
4870 down 125356271
[INFO] Processing file: 3476/3547
4870 down 5470658457
[INFO] Processing file: 3477/3547
4870 down 5052485540
[INFO] Processing file: 3478/3547
4870 down 229615681
[INFO] Processing file: 3479/3547
4870 down 724437005
[INFO] Processing file: 3480/3547
4870 down 335145783
[INFO] Processing file: 3481/3547
4870 down 65867193
[INFO] Processing file: 3482/3547
4870 down 1253807082
[INFO] Processing file: 3483/3547
4870 down 453886324
[INFO] Processing file: 3484/3547
4870 down 866683589
[INFO] Processing file: 3485/3547
4870 down 1290306900
[INFO] Processing file: 3486/3547
4870 down 41606
[INFO] Processing file: 3487/3547
4870 down 1188788203
[INFO] Processing file: 3

In [14]:
label_list = [label for label in sorted(os.listdir(new_dataset_path)) if os.path.isdir(os.path.join(new_dataset_path, label)) and label[0] != "_"]
label_map = {idx: label for idx, label in enumerate(label_list)}

with open(os.path.join(new_dataset_path, "training_list.txt"), "w+") as f:
    f.write("\n".join(train_list))

with open(os.path.join(new_dataset_path, "validation_list.txt"), "w+") as f:
    f.write("\n".join(val_list))

with open(os.path.join(new_dataset_path, "testing_list.txt"), "w+") as f:
    f.write("\n".join(test_list))

with open(os.path.join(new_dataset_path, "label_map.json"), "w+") as f:
    json.dump(label_map, f)

In [15]:
print("Train samples:", len(train_list))
print("Val samples:", len(val_list))
print("Test samples:", len(test_list))

Train samples: 77292
Val samples: 12600
Test samples: 1050


In [16]:
for i, command in enumerate(commands,1):
    wavPaths = list(paths.list_files(os.path.join(new_dataset_path, command), validExts="wav"))
    print("{}. Command: {}, samples: {}".format(i, command, len(wavPaths)))

1. Command: backward, samples: 2406
2. Command: forward, samples: 2478
3. Command: right, samples: 2586
4. Command: left, samples: 2658
5. Command: down, samples: 2514
6. Command: up, samples: 2586
7. Command: go, samples: 2622
8. Command: stop, samples: 2694
9. Command: on, samples: 2622
10. Command: off, samples: 2550
11. Command: yes, samples: 2658
12. Command: no, samples: 2658
13. Command: learn, samples: 2550
14. Command: follow, samples: 2442
15. Command: zero, samples: 2622
16. Command: one, samples: 2586
17. Command: two, samples: 2550
18. Command: three, samples: 2550
19. Command: four, samples: 2550
20. Command: five, samples: 2694
21. Command: six, samples: 2586
22. Command: seven, samples: 2658
23. Command: eight, samples: 2550
24. Command: nine, samples: 2514
25. Command: bed, samples: 2442
26. Command: bird, samples: 2586
27. Command: cat, samples: 2694
28. Command: dog, samples: 2694
29. Command: happy, samples: 2658
30. Command: house, samples: 2694
31. Command: read, 

In [17]:
test_dataset_path = 'Keyword-MLP/test_data_tt'

for command in commands:
    command_path = os.path.join(test_dataset_path, command)
    
    if not os.path.exists(command_path):
        os.makedirs(command_path)
        
for test_path in test_list:
    command, wav_file = test_path.split('/')[-2:]
    source_path = os.path.join(new_dataset_path, command, wav_file)
    destination_path = os.path.join(test_dataset_path, command, wav_file)
    shutil.copy(source_path, destination_path)